Student Name: Joel Amarou Heuer

Student ID: 202102201

# Imports

In [1]:
import numpy as np
import pandas as pd
import random

# (Hyper)Parameters

In [2]:
# TODO Hyperparameters
# input_size = 
output_size = 10 # -> number of neurons in output-layer (= for each possible rating?)
# [hidden_layers_sizes] = 
# learning_rate = 
number_of_epochs = 1 # how is every sample used for pass/forward/backward

# Parameters
train_data_path = "./data/drugLibTrain_raw.tsv" # please use relative path like this
test_data_path = "./data/drugLibTest_raw.tsv" # please use relative path like this

In [ ]:
def train(train_data, train_labels, valid_data, valid_labels):

	for epoch in range(number_of_epochs):
		index = 0

		# Same thing about [hidden_layers] mentioned above is valid here also
		for data, labels in zip(train_data, train_labels):
			predictions, [hidden_layers] = forward_pass(data)
			loss_signals = derivation_of_loss_function(labels, predictions)
			backward_pass(data, [hidden_layers], predictions, loss_signals)
			loss = loss_function(labels, predictions)

			if index % 2000 == 0: # at each 2000th sample, we run validation set to see our model's improvements
				accuracy, loss = test(valid_data, valid_labels)
				print("Epoch= "+str(epoch)+", Coverage= %"+ str(100*(index/len(train_data))) + ", Accuracy= "+ str(accuracy) + ", Loss= " + str(loss))

			index += 1

	return losses

# Main

In [13]:
# Load Data
input_features =  'commentsReview'

train_data = pd.read_csv(train_data_path, sep='\t')
test_data = pd.read_csv(test_data_path, sep='\t')

train_x = train_data[input_features]
train_y = train_data['rating']
test_x = test_data[input_features]
test_y = test_data['rating']


# ONE-HOT for classes 1-10
# creating one-hot vector notation of labels. (Labels are given numeric in the dataset)
new_train_y = np.zeros(shape=(len(train_y), output_size))
new_test_y = np.zeros(shape=(len(test_y), output_size))

# add 1's into one-hot-matrix
for i in range(len(train_y)) :    
    new_train_y[i][train_y[i]-1] = 1

for i in range(len(test_y)):
    new_test_y[i][test_y[i]-1] = 1

train_y = new_train_y
test_y = new_test_y

# Split training data into [a] Training (75%) and [b] validation (25%)
valid_x = np.asarray(train_x[int(0.75*len(train_x)):-1])
valid_y = np.asarray(train_y[int(0.75*len(train_y)):-1])
train_x = np.asarray(train_x[0:int(0.75*len(train_x))])
train_y = np.asarray(train_y[0:int(0.75*len(train_y))])

# TODO
#train(train_x, train_y, valid_x, valid_y)
#print("Test Scores:")
#print(test(test_x, test_y))

((1, 'a'), (2, 'b'), (3, 'c'))